In [2]:
import pandas as pd
import numpy as np
import pickle
import math

In [2]:
# Funciones útiles

def traductor(s):
    '''
    Convierte string a formato
    utilizable
    '''
    if isinstance(s, str):
        a,b = 'áéíóúüñÁÉÍÓÚÜÑ','aeiouunAEIOUUN'
        trans = str.maketrans(a,b)
        s = s.translate(trans).lower()
        s = s.strip(".; ").split()
        s = " ".join(s)
    return s

def agregar_elem_dict(elem, cod, decod, val):
    '''
    Agrega elemento solo si no está en diccionario
    Si está a la llave elem se le asigna el valor val + 1
    y se retorna el nuevo valor de val y True
    
    '''
    booleano = False
    if elem not in cod:
        val = val + 1
        cod[elem] = val
        decod[val] = elem
        booleano = True
    return val, booleano

# Procesamiento `arbol.xlsx`

In [3]:
# Cargado árbol de especialidades
arbol = pd.read_excel('informacion/arbol.xlsx', header=19)
arbol.columns = ['servicio', 'promedio hs', 'minimo hs', 'maximo hs', 'promedio s', 'minimo s', 'maximo s', 'n1', 'n2', 'n3', 'n4']
arbol = arbol.applymap(traductor)

In [4]:
# Identificación de errores
errores_arbol = arbol[arbol[['servicio', 'promedio hs', 'minimo hs', 'maximo hs', 'promedio s', 'minimo s', 'maximo s']].isna().any(axis=1)]

# Extracción filas con información incompleta 
arbol.dropna(subset=['servicio', 'promedio hs', 'minimo hs', 'maximo hs', 'promedio s', 'minimo s', 'maximo s'], inplace=True)

display(errores_arbol)

,servicio,promedio hs,minimo hs,maximo hs,promedio s,minimo s,maximo s,n1,n2,n3,n4
0,asesoria tributaria general,NaN,NaN,NaN,NaN,NaN,NaN,derecho tributario,asesoria tributaria local,asesoria tributaria empresas,NaN


In [5]:
# Codificación, lista de padres y tabla de servicios
codificacion = {}
decodificacion = {}
padres = []
val = -1
cont = 0
servicios = pd.DataFrame(columns=['id', 'promedio hs', 'minimo hs', 'maximo hs', 'promedio s', 'minimo s', 'maximo s'])
for index, row in arbol.iterrows():
    s = row['servicio']
    val, booleano = agregar_elem_dict(s, codificacion, decodificacion, val)
    if booleano:
        l = [val] + list(row[['promedio hs', 'minimo hs', 'maximo hs', 'promedio s', 'minimo s', 'maximo s']])
        servicios.loc[cont] = l
        cont += 1 
        padres.append(-1)
    rama = list(row[['n1', 'n2', 'n3', 'n4', 'servicio']].dropna())
    rama.reverse()
    for i in range(1, len(rama)):
        rama[i] = f'n{len(rama) - i}' + rama[i]
        val, booleano = agregar_elem_dict(rama[i], codificacion, decodificacion, val)
        padres[codificacion[rama[i-1]]] = codificacion[rama[i]]
        if booleano:
            padres.append(-1)
        else:
            break
servicios['id'] = servicios['id'].astype(int)

In [6]:
# Guardamos objetos creados
file = open('codificacion.pickle', 'wb')
pickle.dump(codificacion, file)
file.close()

file2 = open('servicios.pickle', 'wb')
pickle.dump(servicios, file2)
file2.close()

file3 = open('padres.pickle', 'wb')
pickle.dump(padres, file3)
file3.close()

file4 = open('decodificacion.pickle', 'wb')
pickle.dump(decodificacion, file4)
file4.close()

In [7]:
########### EJECUTAR ESTO PARA NUEVA SESIONES #############

# Cargamos objetos
file = open('codificacion.pickle', 'rb')
codificacion = pickle.load(file)
file.close()

file2 = open('servicios.pickle', 'rb')
servicios = pickle.load(file2)
file2.close()

file3 = open('padres.pickle', 'rb')
padres = pickle.load(file3)
file3.close()

file4 = open('decodificacion.pickle', 'rb')
decodificacion = pickle.load(file4)
file4.close()

# Procesamiento `casos.xlsx`

In [8]:
# Cargado de casos
archivo_casos = pd.read_excel('informacion/casos.xlsx')
archivo_casos = list(archivo_casos.iloc[4:,0])

In [9]:
# Generación lista de casos con servicios asociados
casos = []
cont = 0
largo = len(archivo_casos)
errores = set()
while cont < largo:
    caso = []
    while cont < largo:
        serv = archivo_casos[cont]
        if serv is np.nan:
            cont += 3
            break
        try:
            caso.append(codificacion[traductor(serv)])
        except KeyError:
            errores.add(traductor(serv))
        cont += 1
    casos.append(caso)

In [10]:
# Guardamos casos
file = open('casos.pickle', 'wb')
pickle.dump(casos, file)
file.close()

In [11]:
########### EJECUTAR ESTO PARA NUEVA SESIONES #############

# Cargamos casos
file = open('casos.pickle', 'rb')
casos = pickle.load(file)
file.close()

# Procesamiento `abogados.xlsx`

In [12]:
# Cargado excel abogados
abogados = pd.read_excel('informacion/abogados.xlsx', header=4)
abogados.drop(columns=abogados.columns[0], inplace=True)
abogados.columns = ['nombre', 'disponibilidad hs','exp', 'n1', 'n2', 'n3', 'n4', 'servicios', 'servicio', 'nveces', 'calificacion']
abogados = abogados.applymap(traductor)

In [13]:
# Abogados sin horas disponibles declaradas o con información faltante
errores_abogados = abogados[abogados['exp'].isna()]

# Actualizamos lista de abogados sacando aquellos con información faltante
abogados = abogados[~abogados['exp'].isna()]
abogados = abogados[~abogados['servicios'].isna()]

display(errores_abogados)

,nombre,disponibilidad hs,exp,n1,n2,n3,n4,servicios,servicio,nveces,calificacion
13,vicente burgos,3.0,NaN,derecho inmobiliario,bienes raices urbanos; bienes raices rurales,NaN,NaN,compraventa de inmuebles urbanos; compraventa ...,NaN,NaN,NaN


In [14]:
cod_nombres = {}
decod_nombres = {}
tabla_abogados = pd.DataFrame(columns=['id', 'hb', 'exp', 'areas', 'declarados', 'realizados', 'cant', 'promedio'])
idx = 0
for index, row in abogados.iterrows():
    # codifico nombre abogado
    cod_nombres[row['nombre']] = idx
    
    # y guardo decodificacion
    decod_nombres[idx] = row['nombre']
   
    # construccion areas y declarados
    declarados = set()
    areas = set()
    servicios = row['servicios'].split(";")
    for s in servicios:
        if traductor(s) == '':
            continue
        try:
            cod = codificacion[traductor(s)]
        except KeyError:
            continue
        declarados.add(cod)
        padre = padres[cod]
        while True:
            p = padres[padre]
            if p != -1:
                declarados.add(padre)
                padre = p
            else:
                areas.add(padre)
                break
    declarados = list(declarados)
    areas = list(areas)
                
    # construccion realizados, cant y promedio
    realizados = []
    cant = []
    promedio = []
    if not isinstance(row['servicio'], float):
        realizados = row['servicio'].split(';')
        realizados = [codificacion[traductor(r)] for r in realizados]
    
        cant = str(row['nveces']).split(';')
        cant = [int(float(c)) for c in cant]
    
        promedio = str(row['calificacion']).split(';')
        promedio = [float(p) for p in promedio]
    
    # Aumentamos la disponibilidad en un 30%
    fila = [idx, row['disponibilidad hs'] * 1.3, row['exp'], areas, declarados, realizados,
            cant, promedio]
    tabla_abogados.loc[idx] = fila
    idx += 1

In [15]:
# Guardamos objetos creados
file = open('cod_nombres.pickle', 'wb')
pickle.dump(cod_nombres, file)
file.close()

file2 = open('abogados.pickle', 'wb')
pickle.dump(tabla_abogados, file2)
file2.close()

file3 = open('decod_nombres.pickle', 'wb')
pickle.dump(decod_nombres, file3)
file3.close()

In [3]:
########### EJECUTAR ESTO PARA NUEVA SESIONES #############

# Cargamos objetos
file = open('cod_nombres.pickle', 'rb')
cod_nombres = pickle.load(file)
file.close()

file2 = open('abogados.pickle', 'rb')
abogados = pickle.load(file2)
file2.close()

file3 = open('decod_nombres.pickle', 'rb')
decod_nombres = pickle.load(file3)
file3.close()

In [7]:
abogados

,id,hb,exp,areas,declarados,realizados,cant,promedio
0,0,26.0,10.0,[119],"[128, 129, 130, 131, 132, 133, 134, 135, 136, ...",[],[],[]
1,1,13.0,6.0,[119],"[128, 129, 130, 131, 132, 133, 134, 135, 136, ...",[],[],[]
2,2,3.9,9.0,[119],"[128, 129, 130, 131, 132, 133, 134, 135, 136, ...",[],[],[]
3,3,6.5,10.0,"[74, 52, 94, 63]","[66, 67, 68, 70, 72, 73, 106, 107, 108, 112, 1...",[],[],[]
4,4,13.0,8.0,"[74, 52, 94, 63]","[66, 98, 68, 70, 72, 73, 75, 79, 82, 51, 90, 9...",[],[],[]
5,5,26.0,9.0,"[74, 94, 63]","[96, 97, 98, 67, 66, 68, 70, 72, 73, 106, 107,...",[],[],[]
6,6,6.5,10.0,"[74, 94, 63]","[96, 97, 98, 67, 66, 68, 70, 72, 73, 106, 75, ...",[],[],[]
7,7,3.9,8.0,[221],"[218, 219, 220, 222, 223, 224, 225, 226, 227, ...",[],[],[]
8,8,15.6,6.0,[221],"[224, 225, 226, 227, 228, 229, 231, 218, 219, ...",[],[],[]
9,9,32.5,6.0,"[74, 94, 63]","[96, 97, 98, 67, 66, 68, 70, 72, 73, 106, 107,...",[],[],[]
